In [2]:
from V6_tratamiento import tratamiento, tratamiento2
from utils import *

import pandas as pd
import numpy as np
from feature_engine.imputation import RandomSampleImputer
from sklearn.preprocessing import LabelEncoder
from pycaret.classification import *

In [3]:
df_modelado = pd.read_pickle("./pickles_temp/V6.1.pkl") 
df_modelado.head(1)

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_class_groundwater,source_class_surface,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group,random1,random2
0,69572,6000.0,1,3376,275,1390.0,3477,94,34.938093,-9.856322,...,True,False,True,False,False,False,False,2,719,788


In [4]:
model_setup = setup(data=df_modelado, 
                    target = 'status_group', 
                    )
                

,Description,Value
0,Session id,6966
1,Target,status_group
2,Target type,Multiclass
3,Original data shape,"(59400, 81)"
4,Transformed data shape,"(59400, 81)"
5,Transformed train set shape,"(41580, 81)"
6,Transformed test set shape,"(17820, 81)"
7,Numeric features,29
8,Preprocess,True
9,Imputation type,simple


In [5]:
if 0:
    best_model = compare_models()
else:
    best_model = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8047,0.9020,0.8047,0.8016,0.7943,0.6285,0.6363
1,0.8006,0.9019,0.8006,0.7979,0.7910,0.6212,0.6289
2,0.8184,0.9124,0.8184,0.8157,0.8086,0.6553,0.6624
3,0.7949,0.8999,0.7949,0.7905,0.7857,0.6116,0.6178
4,0.7997,0.8975,0.7997,0.7937,0.7874,0.6180,0.6262
5,0.8042,0.9081,0.8042,0.8012,0.7938,0.6276,0.6354
6,0.8098,0.9060,0.8098,0.8059,0.8010,0.6402,0.6459
7,0.8069,0.9049,0.8069,0.8021,0.7980,0.6349,0.6406
8,0.8112,0.9102,0.8112,0.8085,0.8022,0.6422,0.6491


In [6]:
# Probamos el score de la web
df_target = pd.read_csv('./test set values.csv', sep=',')
df_target = tratamiento(df_target)
df_target = tratamiento2(df_target)
df_target

Este df no tiene status_group


,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,...,source_type_spring,source_class_groundwater,source_class_surface,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,random1,random2
0,50785,0.0,174,1996.0,213,35.290799,-4.059696,834,0,3956,...,False,False,True,False,False,False,False,True,244,857
1,51630,0.0,249,1569.0,220,36.656709,-3.309214,2053,5,2753,...,True,True,False,True,False,False,False,False,841,98
2,17168,0.0,385,1567.0,440,34.767863,-5.004344,10014,0,5440,...,False,False,True,False,False,False,False,True,335,330
3,45559,0.0,221,267.0,261,38.058046,-9.418672,5870,7,2831,...,False,True,False,False,False,False,False,True,981,241
4,49871,500.0,72,1260.0,74,35.006123,-10.950412,5981,7,3582,...,True,True,False,True,False,False,False,False,875,278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,144,34.0,225,38.852669,-6.582841,7498,8,8455,...,False,False,True,True,False,False,False,False,282,872
14846,18990,1000.0,285,816.5,318,37.451633,-5.350428,480,5,5069,...,False,True,False,False,False,True,False,False,942,595
14847,28749,0.0,927,1476.0,659,34.739804,-4.585587,606,0,1868,...,False,False,True,True,False,False,False,False,452,58
14848,33492,0.0,241,998.0,220,35.432732,-10.584159,3713,1,6288,...,False,False,True,True,False,False,False,False,697,881


In [7]:
""" df_predictions = predict_model(best_model, data=df_target)
df_result = df_predictions[['id', 'prediction_label']]
df_result.rename(columns={'prediction_label': 'status_group'}, inplace=True)
df_result.set_index('id', inplace=True) """

" df_predictions = predict_model(best_model, data=df_target)\ndf_result = df_predictions[['id', 'prediction_label']]\ndf_result.rename(columns={'prediction_label': 'status_group'}, inplace=True)\ndf_result.set_index('id', inplace=True) "

In [8]:
def prediccion_final(df_target, best_model):
    """ cat_cols= df_target.select_dtypes(include=['object','category']).columns
    print("cat_cols: ", cat_cols)
    num_cols = df_target.select_dtypes(exclude=['object','category']).columns
    le = LabelEncoder()
    for variable in cat_cols:
        df_target[variable] = le.fit_transform(df_target[variable])
 """
    df_predictions = predict_model(best_model, data=df_target)
    df_result = df_predictions[['id', 'prediction_label']]
    df_result.rename(columns={'prediction_label': 'status_group'}, inplace=True)
    df_result.set_index('id', inplace=True)
    df_result['status_group'].replace([0, 1, 2], ['non functional', 'functional needs repair', 'functional'], inplace=True)
    return (df_result)

df_result = prediccion_final(df_target, best_model)

df_result.to_csv("./resultados/V7_0_tratamiento.csv")
# Da un resultado de 0.7611
# Me sorprende mucho la diferencia entre 0.8029 del modelo retornado por el rf y 0.7611 de la web. ¿Estaré dando el mismo tratamiento a los datos?

In [9]:
# Voy a intentar hacer los siguiente antes de la fecha límite de entrega. 

# Probar VIF
# Probar oversample
# Probar ensembles
# Xgboost

# También podría seguir minando los datos creando y borrando columnas, nuevas variables, imputar nulos de otra forma etc... pero eso llevaría
#   mucho tiempo y el objetivo de la práctica no es sacar la mejor puntuación sino aprender

In [10]:
# Pruevo inflacción de la varianza

# Copiado del archivo "multicolinealidad.ipynb"

from sklearn.linear_model import LinearRegression

def calculateVIF(data):
    features = list(data.columns)
    num_features = len(features)
    
    model = LinearRegression()
    
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    for ite in range(num_features):
        x_features = features[:]
        y_featue = features[ite]
        x_features.remove(y_featue)
        
        x = data[x_features]
        y = data[y_featue]
        
        model.fit(data[x_features], data[y_featue])
        
        result[y_featue] = 1/(1 - model.score(data[x_features], data[y_featue]))
    
    return result

calculateVIF(df_modelado)
df_modelado

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_class_groundwater,source_class_surface,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group,random1,random2
0,69572,6000.0,1,3376,275,1390.0,3477,94,34.938093,-9.856322,...,True,False,True,False,False,False,False,2,719,788
1,8776,0.0,0,1709,53,1399.0,1750,39,34.698766,-2.147466,...,False,True,True,False,False,False,False,2,384,966
2,34310,25.0,1,2413,8,686.0,4397,408,37.460664,-3.821329,...,False,True,False,True,False,False,False,2,770,718
3,67743,0.0,0,3962,1057,263.0,4002,222,38.486161,-11.155298,...,True,False,False,True,False,False,False,0,571,16
4,19728,0.0,0,794,1,1527.0,906,135,31.130847,-1.825359,...,False,True,True,False,False,False,False,2,591,472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,1,1663,610,1210.0,1093,610,37.169807,-3.253847,...,True,False,True,False,False,False,False,2,608,769
59396,27263,4700.0,1,1118,23,1212.0,1184,15,35.249991,-9.070629,...,False,True,True,False,False,False,False,2,128,382
59397,37057,0.0,0,667,1,1532.5,711,1,34.017087,-8.750434,...,True,False,False,False,True,False,False,2,526,296
59398,31282,0.0,0,2535,3,820.5,2852,5,35.861315,-6.378573,...,True,False,False,False,True,False,False,2,558,22


In [11]:
# Dejo los nombres de la copia para no falle nada y no perder tiempo

wine_vif = df_modelado.copy(deep = True)
features = list(wine_vif.columns)
features.remove('status_group')
wine_vif = wine_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = wine_vif[fet]
    y = df_modelado[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)

# inf significa infinito. Parace que TODAS las variable booleanas tienen un indice de inflacción de la varianza infinito.

El valor del VIF para la variable id es: 1.0010078544459629
El valor del VIF para la variable amount_tsh es: 1.0690696166039644
El valor del VIF para la variable amount_tsh_bool es: 3.2888740401529537
El valor del VIF para la variable funder es: 1.3981249263295465
El valor del VIF para la variable funder_frequency_encoded es: 1.3304814177995374
El valor del VIF para la variable gps_height es: 1.9825054705491498
El valor del VIF para la variable installer es: 1.55900519832916
El valor del VIF para la variable installer_frequency_encoded es: 1.459931899000686
El valor del VIF para la variable longitude es: 2.3883630384585657
El valor del VIF para la variable latitude es: 1.9339265127311387
El valor del VIF para la variable wpt_name es: 1.124996679397665
El valor del VIF para la variable wpt_name_frequency_encoded es: 1.157990744387005
El valor del VIF para la variable num_private_b es: 1.1143297934996883
El valor del VIF para la variable basin es: 1.9596304727038047
El valor del VIF para

In [12]:
# Voy a probar la automatización de la eliminación de variables hasta que la mayor sea inferior a 5
def calculateVIF(data):
    features = list(data.columns)
    num_features = len(features)
    
    model = LinearRegression()
    
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    for ite in range(num_features):
        x_features = features[:]
        y_featue = features[ite]
        x_features.remove(y_featue)
        
        x = data[x_features]
        y = data[y_featue]
        
        model.fit(data[x_features], data[y_featue])
        
        result[y_featue] = 1/(1 - model.score(data[x_features], data[y_featue]))
    
    return result

wine_vif = df_modelado.copy(deep = True)
features = list(wine_vif.columns)
features.remove('status_group')
wine_vif = wine_vif[features]

calculateVIF(wine_vif)

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_type_spring,source_class_groundwater,source_class_surface,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,random1,random2
VIF,1.001008,1.06907,3.288874,1.398125,1.330481,1.982505,1.559005,1.459932,2.388363,1.933927,...,inf,inf,inf,inf,inf,inf,inf,inf,1.000852,1.000998


In [13]:
# 7'
# Selecciona las columnas que no presentan colinealidad con otras. 
def selectDataUsingVIF(data, max_VIF = 5):
    result = data.copy(deep = True)
    
    VIF = calculateVIF(result)
    
    while VIF.values.max() > max_VIF:
        col_max = np.where(VIF == VIF.values.max())[1][0]
        features = list(result.columns)
        features.remove(features[col_max])
        result = result[features]
        
        VIF = calculateVIF(result)
        
    return result

selected_data = calculateVIF(selectDataUsingVIF(wine_vif))
columns = selected_data.columns

In [29]:
columns

Index(['id', 'amount_tsh', 'amount_tsh_bool', 'funder',
       'funder_frequency_encoded', 'gps_height', 'installer',
       'installer_frequency_encoded', 'longitude', 'latitude', 'wpt_name',
       'wpt_name_frequency_encoded', 'num_private_b', 'basin', 'subvillage',
       'subvillage_frequency_encoded', 'region', 'region_frequency_encoded',
       'region_code', 'district_code', 'lga', 'lga_frequency_encoded', 'ward',
       'ward_frequency_encoded', 'population', 'public_meeting', 'permit',
       'construction_year', 'year_recorded', 'management_Parastatal',
       'management_Private operator', 'management_VWC', 'management_WUA',
       'management_WUG', 'management_Water Board',
       'management_Water authority', 'extraction_type_class_motorpump',
       'extraction_type_class_other', 'extraction_type_class_rope pump',
       'extraction_type_class_submersible',
       'extraction_type_class_wind-powered', 'management_group_other/unknown',
       'management_group_user-group'

In [14]:
# Pruebo el modelo con las nuevas variables
pd.set_option('display.max_columns', None)

new_df_modelado = df_modelado[columns].copy()
print(new_df_modelado.shape)
print(df_modelado.shape)
# vemos que el modelo se ha reducido en 16 columnas que parece tenian multicolinealidad.
# Pruebo a tirarlo de nuevo:

(59400, 65)
(59400, 81)


In [15]:
columns2 = columns

In [19]:
new_df_modelado.head(1)

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,wpt_name,wpt_name_frequency_encoded,num_private_b,basin,subvillage,subvillage_frequency_encoded,region,region_frequency_encoded,region_code,district_code,lga,lga_frequency_encoded,ward,ward_frequency_encoded,population,public_meeting,permit,construction_year,year_recorded,management_Parastatal,management_Private operator,management_VWC,management_WUA,management_WUG,management_Water Board,management_Water authority,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,management_group_other/unknown,management_group_user-group,payment_type_monthly,payment_type_on failure,payment_type_other/unknown,payment_type_per bucket,quality_group_fluoride,quality_group_milky,quality_group_salty,quality_group_unknown,quantity_enough,quantity_insufficient,quantity_seasonal,source_type_dam,source_type_other,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2
0,69572,6000.0,1,3376,275,1390.0,3477,94,34.938093,-9.856322,39031,1,0,1,11937,6,3,5294,11,5,51,564,1426,36,109,True,False,1999.0,2011,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,719,788


In [23]:
df_training_set_labels = pd.read_csv("training set labels.csv") 
new_df_modelado = pd.merge(new_df_modelado, df_training_set_labels, how = 'inner', on = 'id')
new_df_modelado['status_group'].replace(['non functional', 'functional needs repair', 'functional'], [0, 1, 2], inplace=True)
new_df_modelado.head(1)


,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,wpt_name,wpt_name_frequency_encoded,num_private_b,basin,subvillage,subvillage_frequency_encoded,region,region_frequency_encoded,region_code,district_code,lga,lga_frequency_encoded,ward,ward_frequency_encoded,population,public_meeting,permit,construction_year,year_recorded,management_Parastatal,management_Private operator,management_VWC,management_WUA,management_WUG,management_Water Board,management_Water authority,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,management_group_other/unknown,management_group_user-group,payment_type_monthly,payment_type_on failure,payment_type_other/unknown,payment_type_per bucket,quality_group_fluoride,quality_group_milky,quality_group_salty,quality_group_unknown,quantity_enough,quantity_insufficient,quantity_seasonal,source_type_dam,source_type_other,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2,status_group_x,status_group_y,status_group
0,69572,6000.0,1,3376,275,1390.0,3477,94,34.938093,-9.856322,39031,1,0,1,11937,6,3,5294,11,5,51,564,1426,36,109,True,False,1999.0,2011,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,719,788,functional,functional,2


In [28]:
new_df_modelado.shape

(59400, 68)

In [25]:
model_setup = setup(data=new_df_modelado, 
                    target = 'status_group', 
                    )

,Description,Value
0,Session id,6699
1,Target,status_group
2,Target type,Multiclass
3,Original data shape,"(59400, 68)"
4,Transformed data shape,"(59400, 72)"
5,Transformed train set shape,"(41580, 72)"
6,Transformed test set shape,"(17820, 72)"
7,Numeric features,29
8,Categorical features,2
9,Preprocess,True


In [26]:
if 0:
    best_model = compare_models()
else:
    best_model = create_model('rf')
# Y los resultados salen mal. Un accuracy de 0 no es realista. Me gusatría indagar en qué sucede si quiero las columnas con colinealidad pero
# quiero probar otras cosas. Continuo en el V7.1

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1510
ridge,Ridge Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.2470
rf,Random Forest Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.8520
qda,Quadratic Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.4040
ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.5550
gbc,Gradient Boosting Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,9.6830
et,Extra Trees Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3790
xgboost,Extreme Gradient Boosting,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.7680
lightgbm,Light Gradient Boosting Machine,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.8420
nb,Naive Bayes,0.9523,0.0000,0.9523,0.9532,0.9524,0.9136,0.9140,0.1740
